In [2]:
import pandas as pd
from datetime import datetime, date
import numpy as np
from sklearn import ensemble, model_selection, preprocessing
import matplotlib.pyplot as plt
import operator

## Import Data

In [3]:
def importData(filename):
    data = pd.read_csv(filename, sep='\t', low_memory=False)
    return data

In [4]:
data06 = importData('citations2006.tsv')
data07 = importData('citations2007.tsv')
data08 = importData('citations2008.tsv')
data09 = importData('citations2009.tsv')
data10 = importData('citations2010.tsv')
data11 = importData('citations2011.tsv')
data12 = importData('citations2012.tsv')
data13 = importData('citations2013.tsv')
data14 = importData('citations2014partial.tsv') # Partial

In [5]:
data = pd.concat([data06, data07, data08, data09,
                 data10, data11, data12, data13, data14]).reset_index()

In [6]:
len(data)

1024303

## Exploring Features

In this section, we will explore the basic features provided in the dataset.

**Description of Features:**

* Created
    * * `index`: original index of entry in datasets separated by years. Only present for ID purposes; non-substantive.

* Citation Information (information about the citation itself)
    * `CitationNumber`: ID number assigned by Philadelphia government; each citation has a unique `CitationNumber`.
    * `FiledDate`: date the citation was processed in the appropriate office.
    * `IssueDate`: date the citation was issued.  
    * `ViolationCode`: [some violation codes](http://william.e.jones.tripod.com/law/traffic_violations.htm). Haven't been able to find a full list of violation codes.
    * `ViolationTitle`: description of the violation.
    * `Location`: where the citation was issued.
    * `Agency`: the agency that issued the citation (e.g., Penn police, Drexel police, SEPTA, etc.)
    * `HearingDate`: date of hearing.
    * `HearingTime`: time of hearing.
* Defendant Information (information about the defendant)
    * `DefendantLastName`: last name of the defendant.  
    * `DefendantFirstName`: first name of the defendant.
    * `DefCity`: defendant's city of residence.
    * `DefState`: defendant's state of residence.
    * `DefZip`: defendant's zip code.
    * `DOB`: defendant's DOB.
    * `Sex`: defendant's sex.
* Outcome information (information about the outcome of the citation)
    * `AmountDue`: fines charged to defendant.
    * `AmountPaid`: fines paid by the defendant.
    * `ClosingDisposition`: judge's decision (guilty, not guilty, etc.)
    * `DispoDate`: date disposition was issued.
    * `JudgeAdmin`: name of judge presiding

In [62]:
data.columns

Index(['index', 'CitationNumber', 'FiledDate', 'IssueDate', 'ViolationCode',
       'ViolationTitle', 'Location', 'Agency', 'DefendantLastName',
       'DefendantFirstName', 'DefCity', 'DefState', 'DefZip', 'DOB', 'Sex',
       'AmountDue', 'AmountPaid', 'ClosingDisposition', 'DispoDate',
       'JudgeAdmin', 'OwnerLastName', 'OwnerFirstName', 'OwnerCity',
       'OwnerState', 'OwnerZip', 'HearingDate', 'HearingTime'],
      dtype='object')

In [73]:
data.iloc[0]['OwnerLastName']

' '

## Cleaning Features

In [7]:
data.columns

Index(['index', 'CitationNumber', 'FiledDate', 'IssueDate', 'ViolationCode',
       'ViolationTitle', 'Location', 'Agency', 'DefendantLastName',
       'DefendantFirstName', 'DefCity', 'DefState', 'DefZip', 'DOB', 'Sex',
       'AmountDue', 'AmountPaid', 'ClosingDisposition', 'DispoDate',
       'JudgeAdmin', 'OwnerLastName', 'OwnerFirstName', 'OwnerCity',
       'OwnerState', 'OwnerZip', 'HearingDate', 'HearingTime'],
      dtype='object')

In [8]:
for feature in data.columns:
    num_null = len(data[data[feature].isnull() == True])
    print(feature + ':', num_null, 'NaNs')

index: 0 NaNs
CitationNumber: 0 NaNs
FiledDate: 0 NaNs
IssueDate: 0 NaNs
ViolationCode: 0 NaNs
ViolationTitle: 0 NaNs
Location: 0 NaNs
Agency: 0 NaNs
DefendantLastName: 11 NaNs
DefendantFirstName: 11 NaNs
DefCity: 0 NaNs
DefState: 2 NaNs
DefZip: 0 NaNs
DOB: 0 NaNs
Sex: 0 NaNs
AmountDue: 0 NaNs
AmountPaid: 0 NaNs
ClosingDisposition: 0 NaNs
DispoDate: 0 NaNs
JudgeAdmin: 0 NaNs
OwnerLastName: 12 NaNs
OwnerFirstName: 45 NaNs
OwnerCity: 4 NaNs
OwnerState: 2 NaNs
OwnerZip: 0 NaNs
HearingDate: 0 NaNs
HearingTime: 0 NaNs


The dataset looks pretty good -- not much missing data. Now let's look at possible categories for our target.

In [9]:
data['ClosingDisposition'].drop_duplicates().sort_values().tolist()

[' ',
 '                    ',
 'APPEAL DENIED',
 'APPEAL DENIED       ',
 'APPEAL FAIL TO APPR',
 'APPEAL FAIL TO APPR ',
 'APPEAL GLTY W/AMEND',
 'APPEAL GLTY W/AMEND ',
 'APPEAL GLTY W/REDUCT',
 'APPEAL GUILTY IPP',
 'APPEAL SUSTAINED',
 'APPEAL SUSTAINED    ',
 'APPEAL WITHDRAWN',
 'APPEAL WITHDRAWN    ',
 'COMMIT TRIAL SCHD',
 'DECEASED',
 'DECEASED            ',
 'DISMISS IDENTY FRAUD',
 'DISMISSAL',
 'DISMISSAL           ',
 'GLTY IN ABSN/R453/SO',
 'GLTY IN ABSNTIA R453',
 'GUILTY',
 'GUILTY              ',
 'GUILTY CHANGE PLEA',
 'GUILTY CHANGE PLEA  ',
 'GUILTY IN ABSENTIA',
 'GUILTY IN ABSENTIA  ',
 'GUILTY IPP',
 'GUILTY IPP          ',
 'GUILTY PLEA',
 'GUILTY PLEA         ',
 'GUILTY RULE 453',
 'GUILTY RULE 453     ',
 'GUILTY W/AMENDMENT',
 'GUILTY W/AMENDMENT  ',
 'GUILTY W/REDUCTION',
 'GUILTY W/REDUCTION  ',
 'NOT GLTY IN ABSENTIA',
 'NOT GLTY STAT OF LIM',
 'NOT GUILTY',
 'NOT GUILTY          ',
 'NOT GUILTY DECEASED',
 'NUNC APL HEAR DENIED',
 'NUNC APL HEAR GRNTED

Looks like a number of the categories are repeated with varying numbers of spaces, so let's clean that up:

In [10]:
def replaceValues(df, values):
    
    df2 = df.copy()
 
    for k, v in values.items():
        print('Replacing', v, 'with', k + '...')
        df2.replace(to_replace=v, value=k, inplace=True)
        
    return df2

In [11]:
replace_dict = {'NaN': ' ',
                'APPEAL DENIED': 'APPEAL DENIED       ', 
                'APPEAL FAIL TO APPR': 'APPEAL FAIL TO APPR ', 
                'APPEAL GLTY W/AMEND': 'APPEAL GLTY W/AMEND ', 
               'APPEAL SUSTAINED': 'APPEAL SUSTAINED    ',
               'APPEAL WITHDRAWN': 'APPEAL WITHDRAWN    ',
               'DECEASED': 'DECEASED            ',
               'DISMISSAL': 'DISMISSAL           ',
               'GUILTY': 'GUILTY              ',
               'GUILTY CHANGE PLEA': 'GUILTY CHANGE PLEA  ',
               'GUILTY IN ABSENTIA': 'GUILTY IN ABSENTIA  ',
               'GUILTY IPP': 'GUILTY IPP          ',
               'GUILTY PLEA': 'GUILTY PLEA         ',
               'GUILTY RULE 453': 'GUILTY RULE 453     ',
               'GUILTY W/AMENDMENT': 'GUILTY W/AMENDMENT  ',
               'GUILTY W/REDUCTION': 'GUILTY W/REDUCTION  ',
               'NOT GUILTY': 'NOT GUILTY          ',
               'PLEA BARGAIN': 'PLEA BARGAIN        ',
               'PROSECUTION W/DRAWN': 'PROSECUTION W/DRAWN ',
               'VOID TICKET': 'VOID TICKET         ',
               'NaN2': '                    ',}

data2 = replaceValues(data, replace_dict)

Replacing APPEAL GLTY W/AMEND  with APPEAL GLTY W/AMEND...
Replacing   with NaN...
Replacing DISMISSAL            with DISMISSAL...
Replacing NOT GUILTY           with NOT GUILTY...
Replacing DECEASED             with DECEASED...
Replacing PROSECUTION W/DRAWN  with PROSECUTION W/DRAWN...
Replacing GUILTY RULE 453      with GUILTY RULE 453...
Replacing GUILTY IN ABSENTIA   with GUILTY IN ABSENTIA...
Replacing GUILTY PLEA          with GUILTY PLEA...
Replacing VOID TICKET          with VOID TICKET...
Replacing                      with NaN2...
Replacing APPEAL DENIED        with APPEAL DENIED...
Replacing PLEA BARGAIN         with PLEA BARGAIN...
Replacing GUILTY IPP           with GUILTY IPP...
Replacing APPEAL WITHDRAWN     with APPEAL WITHDRAWN...
Replacing GUILTY W/REDUCTION   with GUILTY W/REDUCTION...
Replacing APPEAL SUSTAINED     with APPEAL SUSTAINED...
Replacing GUILTY CHANGE PLEA   with GUILTY CHANGE PLEA...
Replacing GUILTY               with GUILTY...
Replacing GUILTY W/AMEN

In [12]:
data2.replace(to_replace='NaN2', value='NaN', inplace=True)

In [13]:
data2['ClosingDisposition'].drop_duplicates().sort_values().tolist()

['APPEAL DENIED',
 'APPEAL FAIL TO APPR',
 'APPEAL GLTY W/AMEND',
 'APPEAL GLTY W/REDUCT',
 'APPEAL GUILTY IPP',
 'APPEAL SUSTAINED',
 'APPEAL WITHDRAWN',
 'COMMIT TRIAL SCHD',
 'DECEASED',
 'DISMISS IDENTY FRAUD',
 'DISMISSAL',
 'GLTY IN ABSN/R453/SO',
 'GLTY IN ABSNTIA R453',
 'GUILTY',
 'GUILTY CHANGE PLEA',
 'GUILTY IN ABSENTIA',
 'GUILTY IPP',
 'GUILTY PLEA',
 'GUILTY RULE 453',
 'GUILTY W/AMENDMENT',
 'GUILTY W/REDUCTION',
 'NOT GLTY IN ABSENTIA',
 'NOT GLTY STAT OF LIM',
 'NOT GUILTY',
 'NOT GUILTY DECEASED',
 'NUNC APL HEAR DENIED',
 'NUNC APL HEAR GRNTED',
 'NaN',
 'PLEA BARGAIN',
 'PLED GUILTY AT TRIAL',
 'PROSECUT W/DRAWN 147',
 'PROSECUTION W/DRAWN',
 'TIME SERVED FULL RED',
 'VOID TICKET']

In [14]:
data2['ClosingDisposition'].value_counts()

NOT GUILTY              241802
GUILTY IN ABSENTIA      192769
GLTY IN ABSNTIA R453    169376
GUILTY IPP              128967
GUILTY PLEA              79460
NaN                      38879
PROSECUTION W/DRAWN      35034
PLEA BARGAIN             21606
DISMISSAL                19787
APPEAL SUSTAINED         18272
APPEAL GLTY W/AMEND      13642
NOT GLTY IN ABSENTIA     12058
APPEAL FAIL TO APPR      10528
GUILTY                    9954
NOT GLTY STAT OF LIM      4838
APPEAL DENIED             4414
DISMISS IDENTY FRAUD      3499
GUILTY W/AMENDMENT        3022
GUILTY RULE 453           2556
DECEASED                  2198
GUILTY W/REDUCTION        2087
VOID TICKET               1746
NUNC APL HEAR GRNTED      1682
NUNC APL HEAR DENIED      1563
PLED GUILTY AT TRIAL      1288
GUILTY CHANGE PLEA        1273
PROSECUT W/DRAWN 147       880
APPEAL WITHDRAWN           859
APPEAL GLTY W/REDUCT       236
NOT GUILTY DECEASED         13
TIME SERVED FULL RED        11
COMMIT TRIAL SCHD            2
GLTY IN 

We might think that some `ClosingDisposition`s might be the same (e.g. `GLTY IN ABSN/R453/SO` and `GLTY IN ABSNTIA R453`) but we'll think about that later.

Now, we'll create an easier way to refer to `ClosingDisposition`s.

In [15]:
def addEnum(df, category_name):
    
    df2 = df.copy()
    
    enum_name = category_name + '_Enum'
    
    categories = list(df[category_name].drop_duplicates())
    enums = list(range(len(categories)))
    
    enum_dict = dict(zip(categories, enums))
    
    enum_list = [enum_dict[x] for x in df[category_name].tolist()]
    
    df2[enum_name] = enum_list
    
    return df2
    

In [16]:
data3 = addEnum(data2, 'ClosingDisposition')

In [17]:
data3 = addEnum(data3, 'ViolationCode')

In [18]:
data3 = addEnum(data3, 'ViolationTitle')

In [19]:
data3 = addEnum(data3, 'Agency')

In [20]:
data3 = addEnum(data3, 'Sex')

In [21]:
data3 = addEnum(data3, 'JudgeAdmin')

In [22]:
data3.columns

Index(['index', 'CitationNumber', 'FiledDate', 'IssueDate', 'ViolationCode',
       'ViolationTitle', 'Location', 'Agency', 'DefendantLastName',
       'DefendantFirstName', 'DefCity', 'DefState', 'DefZip', 'DOB', 'Sex',
       'AmountDue', 'AmountPaid', 'ClosingDisposition', 'DispoDate',
       'JudgeAdmin', 'OwnerLastName', 'OwnerFirstName', 'OwnerCity',
       'OwnerState', 'OwnerZip', 'HearingDate', 'HearingTime',
       'ClosingDisposition_Enum', 'ViolationCode_Enum', 'ViolationTitle_Enum',
       'Agency_Enum', 'Sex_Enum', 'JudgeAdmin_Enum'],
      dtype='object')

In [23]:
# Convert dates to numbers

def dateToOrd(df, date_column): # Converts dates to ordinal
    
    df2 = df.copy()
    
    new_col = date_column + "_Ordinal"
    date_list = pd.to_datetime(df[date_column], format='%m/%d/%Y', errors='coerce').tolist()
    
    #prob_index = list()
    #for i in range(len(date_list)):
    #    try:
    #       date_list[i].toordinal()
    #    except ValueError:
    #        prob_index += [i]

    df2[new_col] = [x.toordinal() for x in date_list]
    
    return df2
    #return prob_index

In [24]:
data4 = dateToOrd(data3, 'FiledDate')

In [25]:
data4 = dateToOrd(data4, 'IssueDate')

We have a bit of an issue with `HearingDate` because some entries are not present. For now, we'll drop those rows.

In [26]:
data4 = data4[data4['HearingDate'].str.contains('\d')] # drops dates that are not valid

In [27]:
pct = str(round((1-(len(data4)/len(data3)))*100, 2))

print("Percent of rows dropped:", pct + "%")

Percent of rows dropped: 0.66%


In [28]:
data4 = dateToOrd(data4, 'HearingDate')

In [29]:
data4.columns

Index(['index', 'CitationNumber', 'FiledDate', 'IssueDate', 'ViolationCode',
       'ViolationTitle', 'Location', 'Agency', 'DefendantLastName',
       'DefendantFirstName', 'DefCity', 'DefState', 'DefZip', 'DOB', 'Sex',
       'AmountDue', 'AmountPaid', 'ClosingDisposition', 'DispoDate',
       'JudgeAdmin', 'OwnerLastName', 'OwnerFirstName', 'OwnerCity',
       'OwnerState', 'OwnerZip', 'HearingDate', 'HearingTime',
       'ClosingDisposition_Enum', 'ViolationCode_Enum', 'ViolationTitle_Enum',
       'Agency_Enum', 'Sex_Enum', 'JudgeAdmin_Enum', 'FiledDate_Ordinal',
       'IssueDate_Ordinal', 'HearingDate_Ordinal'],
      dtype='object')

### Initial Model

In [30]:
m1df = data4[['ClosingDisposition_Enum', 'ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal', 'IssueDate_Ordinal',
             'HearingDate_Ordinal']]

In [31]:
y_m1df = m1df['ClosingDisposition_Enum']

In [32]:
X_m1df = m1df[['ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal', 'IssueDate_Ordinal',
             'HearingDate_Ordinal']]

In [33]:
X_tr, X_ts, y_tr, y_ts = model_selection.train_test_split(X_m1df, y_m1df, test_size=0.20, random_state=33)

In [34]:
rfc = ensemble.RandomForestClassifier()
rfc.fit(X_tr, y_tr)
print('RFC score:', rfc.score(X_ts, y_ts))

RFC score: 0.571049515264


To put this score in perspective, we can create a "baseline" model: always guessing the most common closing disposition. In this case...

In [35]:
m1df['ClosingDisposition_Enum'].value_counts().sort_values(ascending=False)

0     241802
2     191900
3     168481
7     128737
6      79460
1      35684
10     35034
29     21606
22     19787
5      18272
8      13614
9      12058
12     10470
11      9949
4       4838
14      4402
21      3499
16      3013
15      2555
20      2086
17      1749
27      1682
23      1559
32      1288
24      1272
18       880
19       856
13       765
26       236
31        13
25        11
28         2
30         1
33         1
Name: ClosingDisposition_Enum, dtype: int64

In [36]:
print("Baseline score:", 241802/len(m1df))

Baseline score: 0.23762876365273075


Compared to this baseline score of `0.2376`, our random forests classifier is performing reasonably well (`0.5722`). Let's look at the feature weights.

In [45]:
def getFeatureWeights(features, weights): #Gets feature weights for RFC
    feature_weights_dict = dict(zip(features, weights))
    sorted_feature_weights_dict = sorted(feature_weights_dict.items(), key=operator.itemgetter(1), reverse=True)
    
    print("Feature Weights:")
    
    return sorted_feature_weights_dict

In [ ]:
getFeatureWeights(list(X_tr.columns), list(rfc.feature_importances_))

With standard scaling:

In [196]:
rfc_ss = ensemble.RandomForestClassifier()
rfc_ss.fit(X_tr_ss, y_tr)
print('rfc_ss score:', rfc_ss.score(X_ts_ss, y_ts))

rfc_ss score: 0.572056821923
Feature importances: [ 0.07503941  0.07026839  0.01206139  0.01924478  0.21944553  0.19638459
  0.18325193  0.22430398]


In [ ]:
getFeatureWeights(list(X_tr.columns), list(rfc_ss.feature_importances_))

Is date redundant?

In [39]:
dates = X_m1df[['FiledDate_Ordinal', 'IssueDate_Ordinal', 'HearingDate_Ordinal']]

In [40]:
np.corrcoef(dates, rowvar=False)

array([[ 1.        ,  0.99987709,  0.77971149],
       [ 0.99987709,  1.        ,  0.77958949],
       [ 0.77971149,  0.77958949,  1.        ]])

`FiledDate` and `IssueDate` are virtually completely correlated, so we'll remove one as a feature.

In [41]:
m2df = data4[['ClosingDisposition_Enum', 'ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal',
             'HearingDate_Ordinal']]

X_m2df = m2df[['ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal',
             'HearingDate_Ordinal']]
y_m2df = m2df['ClosingDisposition_Enum']

In [42]:
X_tr, X_ts, y_tr, y_ts = model_selection.train_test_split(X_m2df, y_m2df, test_size=0.20, random_state=33)

In [43]:
rfc3 = ensemble.RandomForestClassifier()
rfc3.fit(X_tr, y_tr)
print('RFC score:', rfc3.score(X_ts, y_ts))

RFC score: 0.56837155366


In [49]:
getFeatureWeights(list(X_tr.columns), list(rfc3.feature_importances_))

Feature Weights:


[('HearingDate_Ordinal', 0.30532733272162565),
 ('FiledDate_Ordinal', 0.29064740801733396),
 ('JudgeAdmin_Enum', 0.23465278767389766),
 ('ViolationCode_Enum', 0.071611823832139529),
 ('ViolationTitle_Enum', 0.068053848663122612),
 ('Sex_Enum', 0.017697196580768425),
 ('Agency_Enum', 0.012009602511112084)]

In [50]:
# Add day of week enum

m3df = data4[['ClosingDisposition_Enum', 'ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal',
             'HearingDate_Ordinal']]

In [53]:
m3df['HearingDate_Day'] = [datetime.fromordinal(x).weekday() for x in m3df['HearingDate_Ordinal']]

/Users/lucy/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [56]:
X_m3df = m3df[['ViolationCode_Enum',
             'ViolationTitle_Enum', 'Agency_Enum', 'Sex_Enum',
             'JudgeAdmin_Enum', 'FiledDate_Ordinal',
             'HearingDate_Ordinal', 'HearingDate_Day']]
y_m3df = m3df['ClosingDisposition_Enum']

In [57]:
X_tr, X_ts, y_tr, y_ts = model_selection.train_test_split(X_m3df, y_m3df, test_size=0.20, random_state=33)

In [58]:
rfc3 = ensemble.RandomForestClassifier()
rfc3.fit(X_tr, y_tr)
rfc3.score(X_ts, y_ts)

0.57196346179359547

In [ ]:
# Feature ideas: Add day of week 

In [54]:
m3df['HearingDate_Day']

0          1
1          1
2          2
3          2
4          2
5          1
6          2
7          2
8          4
9          4
10         4
11         4
12         0
13         4
14         4
15         0
16         0
17         0
18         0
19         2
20         2
21         0
22         0
23         2
24         2
25         2
26         3
27         3
28         3
29         3
          ..
1024273    1
1024274    1
1024275    1
1024276    1
1024277    1
1024278    1
1024279    0
1024280    0
1024281    0
1024282    1
1024283    0
1024284    1
1024285    1
1024286    1
1024287    1
1024288    1
1024289    1
1024290    1
1024291    1
1024292    1
1024293    1
1024294    1
1024295    1
1024296    0
1024297    2
1024298    2
1024299    2
1024300    1
1024301    2
1024302    1
Name: HearingDate_Day, Length: 1017562, dtype: int64